In [1]:
%reset -f
from tkinter import *
from tkinter.ttk import Combobox
from tkinter import messagebox
from tkinter import filedialog
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
import numpy as np
import apt_importers as apt
import math
from matplotlib import gridspec
import re
from tkinter.filedialog import asksaveasfile
import tkinter.scrolledtext as tkst
import webbrowser
from mpl_toolkits.mplot3d import proj3d
import pandas as pd
import h5py

def data_for_cylinder_along_z(center_x,center_y,center_z,radius,height_z):
    z = np.linspace(center_z-height_z/2, center_z+height_z/2, 50)
    theta = np.linspace(0, 2*np.pi, 50)
    theta_grid, z_grid=np.meshgrid(theta, z)
    x_grid = radius*np.cos(theta_grid) + center_x
    y_grid = radius*np.sin(theta_grid) + center_y
    return x_grid,y_grid,z_grid

def gerade(k,x,d):
    return k*x+d

class APT_analyzer():                                                          # define class object 
    def __init__(self,tk):                                                     # initialize when starting tkinter
        self.plot_exist=0
        self.conc_check=0
        self.excess_check=0
        self.invert=0
        self.data_color=['blue','red','green','yellow','purple','black','white','grey','brown'] # colors of cylinder to be seletected
        self.color_cp=['blue','red','green','yellow','purple','black','white','grey','brown']
        self.frame=Frame(tk)                                                   # setting up frame
        self.frame.pack(expand=True,fill=BOTH)                                 # placing frame
        self.canvas_all=Canvas(self.frame,width=1400,height=800,scrollregion=(0,0,1550,850))     #setting up canvas onto frame
        ################# scroll bars vertical and horizontal  ################
        self.hbar=Scrollbar(self.frame,orient=HORIZONTAL)                      # setting up horizontal scrollbar
        self.hbar.pack(side=TOP,fill=X)                                        # placing horizontal scrollbar on the top side
        self.hbar.config(command=self.canvas_all.xview)                        # configurating horizontal scrollbar
        self.vbar=Scrollbar(self.frame,orient=VERTICAL)                        # setting up vertical scrollbar
        self.vbar.pack(side=LEFT,fill=Y)                                       # placing vertical scrollbar on the left side
        self.vbar.config(command=self.canvas_all.yview)                        # configurating vertical scrollbar
        self.canvas_all.config(yscrollcommand=self.vbar.set,xscrollcommand=self.hbar.set)       #configurating canvas
        

        self.b_page1_loading = Button(self.canvas_all, text="1. Load", command=self.page_1,font=('helvetica',12,'bold')) # button for tip calculation
        self.canvas_all.create_window(75,20,window=self.b_page1_loading)
        
        self.b_page2_visualize = Button(self.canvas_all, text="2. Visualize", command=self.page_2,font=('helvetica',12,'bold')) # button for tip calculation
        self.canvas_all.create_window(180,20,window=self.b_page2_visualize)
        
        self.b_page3_cylinder = Button(self.canvas_all, text="3. Calculate", command=self.page_3,font=('helvetica',12,'bold')) # button for tip calculation
        self.canvas_all.create_window(300,20,window=self.b_page3_cylinder)

        ############## initiate values #######################################
        #### page 1 ####  
        self.succ=0
        self.utext0=Label(self.canvas_all,text='Welcome to the APT-analyzer!')    # title of segment
        self.utext1=Label(self.canvas_all,text='This open source tool enables you to look at and analyze APT tips ')    # title of segment
        self.utext2=Label(self.canvas_all,text='All you need is a .pos (or .epos) file and a .rrng file')    # title of segment
        self.utext3=Label(self.canvas_all,text='Select the files you want to look at below and press "Load files"')    # title of segment
        self.utext4=Label(self.canvas_all,text='------------- Initiate files --------------')    # title of segment
        self.b_pos_file = Button(self.canvas_all,bg='green',fg='white', text="Select .pos /.epos file", command=self.search_pos)        # button for pos file initiation
        self.b_rrng_file = Button(self.canvas_all,bg='green',fg='white', text="Select .rrng file", command=self.search_rrng)             # button for rrng file initiation
        self.b_calc_tip = Button(self.canvas_all, text="Load files", command=self.calculate_tip,bg='blue',fg='white',font=('helvetica',14,'bold')) # button for tip calculation
        self.dtext0=Label(self.canvas_all,text='The loading of the files may take some time.')    # title of segment
        self.dtext1=Label(self.canvas_all,text='When "Sucessfully loaded files!" appears underneath the Load button')    # title of segment
        self.dtext2=Label(self.canvas_all,text='then everthing is working as intendet')    # title of segment
        self.dtext3=Label(self.canvas_all,text='and a preview image of the tip should appear on the right')    # title of segment   
        self.dtext4=Label(self.canvas_all,text='You may now go to section 2.Visualize (top middle)')    # title of segment
        self.dtext5=Label(self.canvas_all,text='where you have more options for the visalization')    # title of segment
        self.dtext6=Label(self.canvas_all,text='as well as the ability to set control points and print them')    # title of segment
        self.dtext7=Label(self.canvas_all,text='After that you can go to section 3.Calculate (top right)')    # title of segment
        self.dtext8=Label(self.canvas_all,text='To calculate the concentration profile and excess profile')    # title of segment
        self.dtext9=Label(self.canvas_all,text='of a cylinder area which you can select within the tip')    # title of segment
        self.label_help=Label(self.canvas_all,text='if you need help press this button:')    # title of segment
        self.b_help = Button(self.canvas_all, text="help",bg='orange',fg='black',command=self.help_web,font=('helvetica',14,'bold')) # button for tip calculation
        
        #### page 2 ####
        self.cb_atom = Combobox(tk, values='') 

        self.diss=0
        self.diss2=0
        self.slider_image_size_cv=DoubleVar()
        self.slider_image_size = Scale(self.canvas_all,from_=1,to=5,orient='horizontal',variable=self.slider_image_size_cv,showvalue=0)
        self.label_plot_tip=Label(self.canvas_all,text='------------- plot options ---------------')    # title of segment
        self.label_image_size=Label(self.canvas_all,text='Select image size')                     # label for resolution
        self.label_slider_res=Label(self.canvas_all,text='small')
        self.label_slider_res2=Label(self.canvas_all,text='large')     
        self.slider_image_size.set(3)
        self.b_plot1 = Button(self.canvas_all, text="plot tip", command=self.plot_tip,bg='blue',fg='white',font=('helvetica',14,'bold')) # button for plotting the tip
        self.var_axis=IntVar()
        self.check_axis = Checkbutton(self.canvas_all,text="Hide Axes", onvalue=1, offvalue=0,variable=self.var_axis,font=('helvetica', 10,'bold'))
        self.label_resolution=Label(self.canvas_all,text='Select number of atoms')                     # label for resolution
        self.label_select_atom=Label(self.canvas_all,text='Select atom type')
        self.slider_res_cv=DoubleVar()
        self.label_slider_res3=Label(self.canvas_all,text='small')
        self.label_slider_res4=Label(self.canvas_all,text='large')
        self.slider_res = Scale(self.canvas_all,from_=1,to=5,orient='horizontal',variable=self.slider_res_cv,showvalue=0)
        self.slider_res.set(1)
        self.label_select_atom_size=Label(self.canvas_all,text='Select atom size')
        self.slider_size_cv=DoubleVar()
        self.slider_size = Scale(self.canvas_all,from_=1,to=200,orient='horizontal',variable=self.slider_size_cv,showvalue=0)
        self.label_slider_res5=Label(self.canvas_all,text='small')
        self.label_slider_res6=Label(self.canvas_all,text='large')  
        self.slider_size.set(50)
        self.i_plot=IntVar()
        self.cb_inverse=Checkbutton(self.canvas_all, text='Flip Tip', onvalue=1, offvalue=0, variable=self.i_plot, font=('helvetica', 10,'bold'))   
        self.control=IntVar()
        self.label_control=Checkbutton(self.canvas_all, text='create control points by clicking on screen', onvalue=1, offvalue=0, variable=self.control, font=('helvetica', 10,'bold'))    
        self.text_box = Text(self.canvas_all,height=10, width=35)
        self.b_control_print = Button(self.canvas_all, text="print",bg='green', command=self.save,fg='white',font=('helvetica',12,'bold'))        # button for pos file initiation
        self.b_control_clear = Button(self.canvas_all, text="clear",bg='red', command=self.clear_text,fg='white',font=('helvetica',12,'bold'))        # button for pos file initiation
        self.var_points=IntVar()
        self.check_points = Checkbutton(self.canvas_all,text="plot with points", onvalue=1, offvalue=0,variable=self.var_points,font=('helvetica', 10,'bold'))
        self.b_adjust_cly = Button(self.canvas_all, text="adjust cylinder according to points",bg='yellow',fg='black', command=self.adjust,font=('helvetica',10,'bold'))        # button for pos file initiation     
        self.cb_color_cp = Combobox(tk, values=self.color_cp, width=10) 
        self.cb_color_cp.current(0)
        self.var_plane=IntVar()
        self.label_color_cp=Label(self.canvas_all,text='color of control points')  
        self.check_plane = Checkbutton(self.canvas_all,text="show fit plane and normal vector", onvalue=1, offvalue=0,variable=self.var_plane,font=('helvetica', 10,'bold'))
        
        #### page 3 ####
        self.var_cyl=IntVar()
        self.label_conc=Label(self.canvas_all,text='------------ concentration calculation option  ------------')#.place(x=20,y=400)    # title of segment
        self.var_cyl=IntVar()
        self.check_cylinder = Checkbutton(self.canvas_all,text="show cylinder for concentration calculation", onvalue=1, offvalue=0,variable=self.var_cyl,font=('helvetica', 10,'bold'))#.place(x=20,y=220)
        
        self.b_calc_conc = Button(self.canvas_all, text="plot concentration", command=self.calc_con,bg='blue',fg='white',font=('helvetica',12,'bold'))   # button for calculating the concentration of cylinder
        self.b_calc_excess = Button(self.canvas_all, text="plot excess", command=self.calc_excess,bg='blue',fg='white',font=('helvetica',12,'bold'))    # button for calculating the concentration of cylinder
        self.b_plot_cyl = Button(self.canvas_all, text="plot cylinder", command=self.calc_zoom,bg='blue',fg='white',font=('helvetica',13,'bold'))  # button for calculating the concentration of cylinder
        self.b_print_ex = Button(self.canvas_all, text="print excess", command=self.save_excess,bg='green',fg='white',font=('helvetica',13,'bold'))    # button for calculating the concentration of cylinder
        
        self.label_cly_pos=Label(self.canvas_all,text='cylinder center position')
        self.label_cly_x=Label(self.canvas_all,text='x')
        self.cly_x=StringVar(value='0')
        self.entry_cly_x=Entry(self.canvas_all,textvariable=self.cly_x,width=10)        
        self.label_cly_y=Label(self.canvas_all,text='y') 
        self.cly_y=StringVar(value='0')
        self.entry_cly_y=Entry(self.canvas_all,textvariable=self.cly_y,width=10)        
        self.label_cly_z=Label(self.canvas_all,text='z')
        self.cly_z=StringVar(value='0')
        self.entry_cly_z=Entry(self.canvas_all,textvariable=self.cly_z,width=10)
        self.label_cly_radius=Label(self.canvas_all,text='radius of cylinder')  
        self.radius=StringVar(value='10')
        self.entry_cly_radius=Entry(self.canvas_all,textvariable=self.radius)        
        self.label_cly_height=Label(self.canvas_all,text='height of cylinder') 
        self.height=StringVar(value='30')
        self.entry_cly_height=Entry(self.canvas_all,textvariable=self.height)        
        self.label_cly_beta=Label(self.canvas_all,text='tilt along x-axis')
        self.beta=StringVar(value='0')
        self.entry_cly_beta=Entry(self.canvas_all,textvariable=self.beta)
        self.label_cly_alpha=Label(self.canvas_all,text='tilt along y-axis')
        self.alpha=StringVar(value='0')
        self.entry_cly_alpha=Entry(self.canvas_all,textvariable=self.alpha)
        self.label_color_cyl=Label(self.canvas_all,text='color of the cylinder')   # label for cylinder color
        self.cb_color = Combobox(self.canvas_all, values=self.data_color)
        self.cb_color.current(0)            
        self.label_inter=Label(self.canvas_all,text='Select interval')      # label for resolution
        self.slider_inter_cv=DoubleVar()
        self.label_slider_inter=Label(self.canvas_all,text='small                 large')    # title of segment
        self.slider_inter = Scale(self.canvas_all,from_=1,to=20,orient='horizontal',variable=self.slider_inter_cv,showvalue=0)#Combobox(self.canvas_all, values=self.data_res)                                                            # combobox for resolution
        self.slider_inter.set(15)
        self.b_plot2 = Button(self.canvas_all, text="plot tip", command=self.plot_tip,bg='blue',fg='white',font=('helvetica',13,'bold')) # button for plotting the tip
        self.b_print_conc = Button(self.canvas_all, text="print concentration", command=self.save_conc,bg='green',fg='white',font=('helvetica',13,'bold')) # button for plotting the tip
        self.var_norm=IntVar()
        self.check_norm = Checkbutton(self.canvas_all,text="normalize Excess", onvalue=1, offvalue=0,variable=self.var_norm)
        self.var_switch=IntVar()
        self.check_switch = Checkbutton(self.canvas_all,text="set cylinders                  as bulk*", onvalue=1, offvalue=0,variable=self.var_switch)
        self.label_astrix=Label(self.canvas_all,text='* the conc. of each segment is calculated in relation to the ends')  
        self.data_bulk=['start','end']
        self.cb_bulk = Combobox(tk, values=self.data_bulk, width=4) 
        self.cb_bulk.current(0)
        self.text_box_ex = Text(self.canvas_all,height=6, width=25)
        #self.cb_exp=Checkbutton(self.canvas_all, text='create points for excess calculation', onvalue=1, offvalue=0, variable=self.control, font=('helvetica', 10,'bold'))    
        self.label_exp=Label(self.canvas_all,text='For calcuating the excess select 4 points')  
        
        
        ############## setting up page 1 ##################
        self.a1=self.canvas_all.create_window(200,50,window=self.utext0)        
        self.a2=self.canvas_all.create_window(200,70,window=self.utext1)        
        self.a3=self.canvas_all.create_window(200,90,window=self.utext2)        
        self.a4=self.canvas_all.create_window(200,110,window=self.utext3)        
        self.a5=self.canvas_all.create_window(200,140,window=self.utext4)  
        self.a6=self.canvas_all.create_window(120,165,window=self.b_pos_file) 
        self.a7=self.canvas_all.create_window(280,165,window=self.b_rrng_file) 
        self.a8=self.canvas_all.create_window(200,230,window=self.b_calc_tip)
        self.a9=self.canvas_all.create_window(200,300,window=self.dtext0)        
        self.a10=self.canvas_all.create_window(200,320,window=self.dtext1)  
        self.a11=self.canvas_all.create_window(200,340,window=self.dtext2)   
        self.a12=self.canvas_all.create_window(200,360,window=self.dtext3)   
        self.a13=self.canvas_all.create_window(200,400,window=self.dtext4)        
        self.a14=self.canvas_all.create_window(200,420,window=self.dtext5)        
        self.a15=self.canvas_all.create_window(200,440,window=self.dtext6)
        self.a16=self.canvas_all.create_window(200,460,window=self.dtext7)
        self.a17=self.canvas_all.create_window(200,480,window=self.dtext8)
        self.a18=self.canvas_all.create_window(200,500,window=self.dtext9)
        self.a19=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a20=self.canvas_all.create_window(200,560,window=self.b_help)
        self.a21=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a22=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a23=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a24=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a25=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a26=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a27=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a28=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a29=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a30=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a31=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a32=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a33=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a34=self.canvas_all.create_window(200,520,window=self.label_help)
            
        ###################### setup figures  #################################                        
        self.fig=Figure(figsize=(17,13))                                       # defining figure size
        self.canvas=FigureCanvasTkAgg(self.fig,master=self.canvas_all)         # setting up canvas for figure
        self.toolbar=NavigationToolbar2Tk(self.canvas, self.canvas_all)        # setting up toolbar for figure
        self.canvas_all.create_window(400,10,window=self.canvas.get_tk_widget(),anchor=N+W,tags='canvas') #placing figure canvas on general canvas
        self.canvas_all.pack(expand=True,fill=BOTH)                            # placing general canvas at the end
        self.E=2                                                               # setting up variable to check if .pos/.epos file is selected
        self.R=2                                                               # setting up variable to check if .rrng file is selected       
              
    def page_1(self):
        self.canvas_all.delete(self.a1,self.a2,self.a3,self.a4,self.a5,self.a6,self.a7,self.a8,self.a9,self.a10,self.a11,self.a12,self.a13,self.a14,self.a15,self.a16,self.a17,self.a18,self.a19,self.a20,self.a21,self.a22,self.a23,self.a24,self.a25,self.a26,self.a27,self.a28,self.a29,self.a30,self.a31,self.a32,self.a33,self.a34)        
        if self.succ==1:self.a_succ=self.canvas_all.create_window(200,260,window=self.label_succ) #placing label on canvas
        if self.R==1:self.aRS=self.canvas_all.create_window(280,190,window=self.label_rrng_selected)
        if self.R==0:self.aR=self.canvas_all.create_window(280,190,window=self.label_rrng_selected)
        if self.E==0:self.aE=self.canvas_all.create_window(120,190,window=self.label_pos_selected)
        if self.E==1:self.aES=self.canvas_all.create_window(120,190,window=self.label_pos_selected)
            
        self.a1=self.canvas_all.create_window(200,50,window=self.utext0)        
        self.a2=self.canvas_all.create_window(200,70,window=self.utext1)        
        self.a3=self.canvas_all.create_window(200,90,window=self.utext2)        
        self.a4=self.canvas_all.create_window(200,110,window=self.utext3)        
        self.a5=self.canvas_all.create_window(200,140,window=self.utext4)  
        self.a6=self.canvas_all.create_window(120,165,window=self.b_pos_file) 
        self.a7=self.canvas_all.create_window(280,165,window=self.b_rrng_file) 
        self.a8=self.canvas_all.create_window(200,230,window=self.b_calc_tip)
        self.a9=self.canvas_all.create_window(200,300,window=self.dtext0)        
        self.a10=self.canvas_all.create_window(200,320,window=self.dtext1)  
        self.a11=self.canvas_all.create_window(200,340,window=self.dtext2)   
        self.a12=self.canvas_all.create_window(200,360,window=self.dtext3)   
        self.a13=self.canvas_all.create_window(200,400,window=self.dtext4)        
        self.a14=self.canvas_all.create_window(200,420,window=self.dtext5)        
        self.a15=self.canvas_all.create_window(200,440,window=self.dtext6)
        self.a16=self.canvas_all.create_window(200,460,window=self.dtext7)
        self.a17=self.canvas_all.create_window(200,480,window=self.dtext8)
        self.a18=self.canvas_all.create_window(200,500,window=self.dtext9)
        self.a19=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a20=self.canvas_all.create_window(200,560,window=self.b_help)
        self.a21=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a22=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a23=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a24=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a25=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a26=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a27=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a28=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a29=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a30=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a31=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a32=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a33=self.canvas_all.create_window(200,520,window=self.label_help)
        self.a34=self.canvas_all.create_window(200,520,window=self.label_help)
        
    def page_2(self):
        self.canvas_all.delete(self.a1,self.a2,self.a3,self.a4,self.a5,self.a6,self.a7,self.a8,self.a9,self.a10,self.a11,self.a12,self.a13,self.a14,self.a15,self.a16,self.a17,self.a18,self.a19,self.a20,self.a21,self.a22,self.a23,self.a24,self.a25,self.a26,self.a27,self.a28,self.a29,self.a30,self.a31,self.a32,self.a33,self.a34)        
        if self.R==1:self.canvas_all.delete(self.aRS)
        if self.R==0:self.canvas_all.delete(self.aR)
        if self.E==0:self.canvas_all.delete(self.aE)
        if self.E==1:self.canvas_all.delete(self.aES)
        if self.succ==1: self.canvas_all.delete(self.a_succ)
        ############## setting up buttons, Scales, comboboxes, etc. for plot options ###################    
        self.a1=self.canvas_all.create_window(200,50,window=self.label_plot_tip)        
        self.a2=self.canvas_all.create_window(100,80,window=self.label_image_size)        
        self.a3=self.canvas_all.create_window(30,105,window=self.label_slider_res)        
        self.a4=self.canvas_all.create_window(170,105,window=self.label_slider_res2)        
        self.a5=self.canvas_all.create_window(100,105,window=self.slider_image_size)        
        self.a6=self.canvas_all.create_window(250,300,window=self.b_plot1)        
        self.a7=self.canvas_all.create_window(280,170,window=self.check_axis)        
        self.a8=self.canvas_all.create_window(100,145,window=self.label_resolution)        
        self.a9=self.canvas_all.create_window(280,80,window=self.label_select_atom)          
        self.a10=self.canvas_all.create_window(30,170,window=self.label_slider_res3)        
        self.a11=self.canvas_all.create_window(170,170,window=self.label_slider_res4)        
        self.a12=self.canvas_all.create_window(100,170,window=self.slider_res)        
        self.a13=self.canvas_all.create_window(100,210,window=self.label_select_atom_size)        
        self.a14=self.canvas_all.create_window(30,230,window=self.label_slider_res5)        
        self.a15=self.canvas_all.create_window(170,230,window=self.label_slider_res6)
        self.a16=self.canvas_all.create_window(100,230,window=self.slider_size)        
        self.a17=self.canvas_all.create_window(280,230,window=self.cb_inverse)        
        self.a18=self.canvas_all.create_window(280,110,window=self.cb_atom)             # putting combobox with element selection on canvas
        self.a19=self.canvas_all.create_window(200,350,window=self.label_control)        
        self.a20=self.canvas_all.create_window(200,450,window=self.text_box)        
        self.a21=self.canvas_all.create_window(100,560,window=self.b_control_print)        
        self.a22=self.canvas_all.create_window(300,560,window=self.b_control_clear)        
        self.a23=self.canvas_all.create_window(200,560,window=self.check_points)  
        self.a24=self.canvas_all.create_window(200,600,window=self.b_adjust_cly)
        self.a25=self.canvas_all.create_window(100,310,window=self.cb_color_cp)
        self.a26=self.canvas_all.create_window(100,290,window=self.label_color_cp)
        self.a27=self.canvas_all.create_window(200,630,window=self.check_plane) 
        self.a28=self.canvas_all.create_window(170,230,window=self.label_slider_res6)
        self.a29=self.canvas_all.create_window(170,230,window=self.label_slider_res6)
        self.a30=self.canvas_all.create_window(170,230,window=self.label_slider_res6)
        self.a31=self.canvas_all.create_window(170,230,window=self.label_slider_res6)
        self.a32=self.canvas_all.create_window(170,230,window=self.label_slider_res6)
        self.a33=self.canvas_all.create_window(170,230,window=self.label_slider_res6)
        self.a34=self.canvas_all.create_window(170,230,window=self.label_slider_res6)
    def page_3(self):
        self.canvas_all.delete(self.a1,self.a2,self.a3,self.a4,self.a5,self.a6,self.a7,self.a8,self.a9,self.a10,self.a11,self.a12,self.a13,self.a14,self.a15,self.a16,self.a17,self.a18,self.a19,self.a20,self.a21,self.a22,self.a23,self.a24,self.a25,self.a26,self.a27,self.a28,self.a29,self.a30,self.a31,self.a32,self.a33,self.a34)        
        if self.R==1:self.canvas_all.delete(self.aRS)
        if self.R==0:self.canvas_all.delete(self.aR)
        if self.E==0:self.canvas_all.delete(self.aE)
        if self.E==1:self.canvas_all.delete(self.aES)
        if self.succ==1: self.canvas_all.delete(self.a_succ)
        ###########  setting up buttons, Scales, comboboxes etc. for cylinder selection and concentration calculations  ###################
        self.label_exp=Label(self.canvas_all,text='For calcuating the excess select 4 points') 
        self.entry_cly_x=Entry(self.canvas_all,textvariable=self.cly_x,width=10) 
        self.entry_cly_y=Entry(self.canvas_all,textvariable=self.cly_y,width=10)
        self.entry_cly_z=Entry(self.canvas_all,textvariable=self.cly_z,width=10)
        self.entry_cly_alpha=Entry(self.canvas_all,textvariable=self.alpha)
        self.entry_cly_beta=Entry(self.canvas_all,textvariable=self.beta)
        
        self.a1=self.canvas_all.create_window(200,50,window=self.label_conc)        
        self.a2=self.canvas_all.create_window(200,90,window=self.check_cylinder)        
        self.a3=self.canvas_all.create_window(100,450,window=self.b_calc_conc)        
        self.a4=self.canvas_all.create_window(100,500,window=self.b_calc_excess)        
        self.a5=self.canvas_all.create_window(300,400,window=self.b_plot_cyl)    
        self.a6=self.canvas_all.create_window(300,500,window=self.b_print_ex) 
        self.a7=self.canvas_all.create_window(200,120,window=self.label_cly_pos)    
        self.a8=self.canvas_all.create_window(100,140,window=self.label_cly_x)        
        self.a9=self.canvas_all.create_window(100,170,window=self.entry_cly_x)      
        self.a10=self.canvas_all.create_window(200,140,window=self.label_cly_y)        
        self.a11=self.canvas_all.create_window(200,170,window=self.entry_cly_y)        
        self.a12=self.canvas_all.create_window(300,140,window=self.label_cly_z)        
        self.a13=self.canvas_all.create_window(300,170,window=self.entry_cly_z)
        self.a14=self.canvas_all.create_window(100,200,window=self.label_cly_radius)
        self.a15=self.canvas_all.create_window(100,225,window=self.entry_cly_radius)                       
        self.a16=self.canvas_all.create_window(300,200,window=self.label_cly_height)        
        self.a17=self.canvas_all.create_window(300,225,window=self.entry_cly_height)         
        self.a18=self.canvas_all.create_window(100,255,window=self.label_cly_beta)        
        self.a19=self.canvas_all.create_window(100,280,window=self.entry_cly_beta)        
        self.a20=self.canvas_all.create_window(300,255,window=self.label_cly_alpha)      
        self.a21=self.canvas_all.create_window(300,280,window=self.entry_cly_alpha)             
        self.a22=self.canvas_all.create_window(100,330,window=self.label_color_cyl)        
        self.a23=self.canvas_all.create_window(100,355,window=self.cb_color)        
        self.a24=self.canvas_all.create_window(300,310,window=self.label_inter)        
        self.a25=self.canvas_all.create_window(300,330,window=self.label_slider_inter)        
        self.a26=self.canvas_all.create_window(300,355,window=self.slider_inter)        
        self.a27=self.canvas_all.create_window(100,400,window=self.b_plot2)        
        self.a28=self.canvas_all.create_window(300,450,window=self.b_print_conc)
        self.a29=self.canvas_all.create_window(100,550,window=self.check_norm)
        self.a30=self.canvas_all.create_window(260,550,window=self.check_switch)
        self.a31=self.canvas_all.create_window(220,580,window=self.label_astrix)
        self.a32=self.canvas_all.create_window(280,550,window=self.cb_bulk)
        self.a33=self.canvas_all.create_window(200,610,window=self.label_exp)
        self.a34=self.canvas_all.create_window(200,675,window=self.text_box_ex)
        
    def help_web(self):
        url='https://github.com/areichm/APT_analyzer'
        new=1
        webbrowser.open(url,new=new)
        
    def clear_text(self):    
        self.text_box.delete('1.0', END)
                
    def save(self):
        points=self.text_box.get(1.0,END)
        points=points.split('\n')
        points=[x for x in points if x!='']
        xyz=[]
        for i in range(0,len(points)):
            points_inner=points[i].split(',')
            for j in points_inner:
                m=j.split('=')
                xyz.append(m[1])
        data=np.array(xyz).reshape(int(len(xyz)/3),3)
        label=('x','y','z')      
        df=pd.DataFrame(data,columns=label)
        h5File = "controlpoints.h5"
        df.to_hdf(h5File,'data')
        messagebox.showinfo(title='APT_analyzer',message='Control points saved to hdf5 file: "controlpoints.h5" inside the APT-analyzer folder.')

    def save_conc(self):
        if self.conc_check==0:messagebox.showinfo(title='APT_analyzer',message='please plot concentration first')
        else: 
            self.df_con.to_csv('APT_analyzer_concentration.csv')
            messagebox.showinfo(title='APT_analyzer',message='Control points saved to csv file: "APT_analyzer_concentration.csv" inside the APT-analyzer folder.')
            
    def save_excess(self):
        if self.excess_check==0:messagebox.showinfo(title='APT_analyzer',message='please plot excess first')
        else: 
            self.df_ex.to_csv('APT_analyzer_excess.csv')
            messagebox.showinfo(title='APT_analyzer',message='Control points saved to csv file: "APT_analyzer_excess.csv" inside the APT-analyzer folder.')
                      
    def adjust(self):                   # adjust cylinder according to controlpoints
        self.var_cyl.set(1)
        XX=[]                           # setting up arrays
        YY=[]
        ZZ=[]
        points=self.text_box.get(1.0,END)   # get text from text box
        points=points.split('\n')           # split the text into each row
        points=[x for x in points if x!=''] # remove any rows that dont contain information
        for i in range(0,len(points)):      # loop over all rows
                points_inner=points[i].split(',')   # split each row into parts of each coordinates
                xyz2=[]
                for j in points_inner:               # loop over all values inside row
                    m=j.split('=')                   # isolate actual number
                    xyz2.append(m[1])                # save number
                XX.append(float(xyz2[0]))            # save x,y and z coordinate
                YY.append(float(xyz2[1]))
                if self.i_plot.get()==0:ZZ.append(float(xyz2[2]))
                if self.i_plot.get()==1:ZZ.append(-float(xyz2[2]))      # if inverse tip is selected reverse z coordinate
        
        if len(XX)>=3:                               # if 3 or more control points are chosen
                center_x=round(sum(XX)/len(XX),3)             # calculate center in x, y and z
                center_y=round(sum(YY)/len(YY),3)
                if self.i_plot.get()==0:center_z=round(sum(ZZ)/len(ZZ),3)
                if self.i_plot.get()==1:center_z=round(-sum(ZZ)/len(ZZ),3)
                self.cly_x=StringVar(value=center_x)                    #save the center as coordinates for cylinder
                self.cly_y=StringVar(value=center_y)
                self.cly_z=StringVar(value=center_z)
                tmp_C=[]
                for i in range(len(XX)):
                    tmp_C.append([XX[i],YY[i],ZZ[i]])                              #write x,y and z into one matrix and transpose it below
                C=np.array(tmp_C).T
                svd=np.linalg.svd(C - np.mean(C, axis=1, keepdims=True))           # using singular value decomposition to find fit
                left=svd[0]                                                        # upper line has to substract out the centroid first
                fit=left[:,-1]
                normal=np.array([fit[0],fit[1],fit[2]])
                norm=np.linalg.norm(normal)
                n_n=normal/norm                                                    # select fitted values
                aa=np.arcsin(-n_n[1])
                bb=math.atan2(n_n[0],n_n[2])     # this is done by applying rotation matrix in reverse order onto normal verctor
                                                 # and then solving the equation for alpha and beta
                #al=round(math.degrees(aa),4)    # that rotate the normal vector into x=0 and y=0
                #if normal[2]<0:al=round(math.degrees(-aa),4)
                al=round(math.degrees(aa),4)
                be=round(math.degrees(bb),4)
                self.alpha=StringVar(value=al)                                     # then transforming into degree and  
                self.beta=StringVar(value=be)                                      # saving alpha and beta values into entry field
        else: messagebox.showinfo(title='APT_analyzer',message='Choose at least 3 control points')  # if less than 3 points are selected show message
                
    def search_pos(self):                                                      # function for searching for pos/epos file
        if self.E==0:self.canvas_all.delete(self.aE)
        if self.E==1:self.canvas_all.delete(self.aES)
        self.filename=filedialog.askopenfilename()                             # getting file from search
        S='.POS' in self.filename                                              # checking if .POS is in the filename
        ES='.EPOS' in self.filename                                            # checking if .EPOS is in the filename
        s='.pos' in self.filename                                              # checking if .pos is in the filename
        es='.epos' in self.filename                                            # checking if .epos is in the filename
        if S or s is True:                                                     # if .POS or .pos was selected
            self.label_pos_selected=Label(text='pos file selected')            # create label for pos
            self.aE=self.canvas_all.create_window(120,190,window=self.label_pos_selected) #placing label on canvas
            self.E=0                                                           # set variable for later recognition
            if self.succ==1: self.canvas_all.delete(self.a_succ) 
            self.succ=0
        elif ES or es is True:                                                 # if .EPOS or .epos was selected
            self.label_pos_selected=Label(text='epos file selected')           # creating label for epos
            self.aES=self.canvas_all.create_window(120,190,window=self.label_pos_selected) #put label on canvas
            self.E=1                                                           # set variable for later recognition
            if self.succ==1: self.canvas_all.delete(self.a_succ) 
            self.succ=0
        else:                                                                  # if any other filetype was selected
            messagebox.showinfo(title='APT_analyzer',message='select .pos or .epos file. Warning: Notation has to be either .pos /.epos or .POS / .EPOS')
            self.filename=[]                                                   # leave filename empty
            self.E=2                                                           # set variable back to 2
            
    def search_rrng(self):                                                     # function for searching for rrng file
        if self.R==1:self.canvas_all.delete(self.aRS)
        if self.R==0:self.canvas_all.delete(self.aR)
        self.filename2=filedialog.askopenfilename()                            # getting file from search
        S='.RRNG' in self.filename2                                            # checking if .RRNG is in the filename
        s='.rrng' in self.filename2                                            # checking if .rrng is in the filename
        SR='.RNG' in self.filename2
        sr='.rng' in self.filename2
        if S or s is True:                                                     # if .RRNG or .rrng was selected
            self.label_rrng_selected=Label(text='.rrng file selected')         # create label for rrng
            self.aR=self.canvas_all.create_window(280,190,window=self.label_rrng_selected) #placing label on canvas
            self.R=0                                                           # setting variable for later recognition
            if self.succ==1: self.canvas_all.delete(self.a_succ) 
            self.succ=0
        elif SR or sr is True:
            self.label_rrng_selected=Label(text='.rng file selected')         # create label for rrng
            self.aRS=self.canvas_all.create_window(280,190,window=self.label_rrng_selected) #placing label on canvas
            self.R=1 
            if self.succ==1: self.canvas_all.delete(self.a_succ) 
            self.succ=0
        else:                                                                  # if no correct file was selected
            messagebox.showinfo(title='APT_analyzer',message='select .rrng file. Warning: Notation has to be either .rrng or .RRNG')
            self.filename=[]                                                   # leaving filename empty
            self.R=2                                                           # set variable back to 2
        
    def calculate_tip(self):                                                   # function for reading and converting the pos and rrng file and creating the tip
        if self.E==2:messagebox.showinfo(title='APT_analyzer',message='no file .pos/.epos file selected')   # checking if no  pos/epos file is selected (E=2)
        elif self.R==2:messagebox.showinfo(title='APT_analyzer',message='no file .rrng file selected')    # checking if no  rrng file is selected (R=2)
        else:                                                                    # if pos/epos and rrng file is selected
            with open(self.filename, 'rb') as f:                                   # opening pos/epos data from file1
                data = f.read()                                                    # reading pos/epos data from file1
            data_rrng=self.filename2                                               # reading rrng data from file2
            if self.E==0:pos=apt.read_pos(data)                                    # converting pos data if pos file was selected (E==0)
            elif self.E==1:pos=apt.read_epos(data)                                 # converting epos data if epos file was selected (E==1)
            if self.R==0:rrngs=apt.read_rrng(data_rrng)                           # converting rrng data
            elif self.R==1:rrngs=apt.read_rng(data_rrng)
            self.tip=apt.label_ions(pos, rrngs)                                    # label ions using apt_importers functions
            self.ele=apt.unique(list(self.tip['comp']))                            # determining how many unique elements in tip using apt_importers functions
            string=['all']                                                         # setting up strings for atom selection combobox
            for i in range (0,len(self.ele)):                                      # loop over all unique elements
                string.append(self.ele[i])                                         # creating strings with all unique elements
            self.data_atom=(string)                                                # data for combobox containing all unique elements + 'all' at start
            self.cb_atom = Combobox(tk, values=self.data_atom)                     # creating combobox with unique elements data 
            self.atom=1
        
            self.cb_atom.current(0)                                                # setting initial value of combobox to 'all'  
            self.fig.clear()                                                   # if plot single was selected clear all previous plots
            self.canvas_all.delete('message') 
            self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[2,1],wspace=0.5,hspace=0.5,height_ratios=[2,1]) #adjust the size of the figure compared to (111)
            self.ax=self.fig.add_subplot(self.spec[0],projection='3d')
            N=5000
            u_ele=self.ele
            x=np.array(self.tip['x'])[::N]                                         # select x cooridant (only take every Nth atom to plot later)
            y=np.array(self.tip['y'])[::N]                                         # select y cooridant from tip data
            z=np.array(self.tip['z'])[::N]                                         # select z cooridant from tip data
            c=np.array(self.tip['colour'])[::N]                                    # select the colour column in the tip data
            label=np.array(self.tip['comp'])[::N]                                  # select the composition column in the tip data
            for i in range(0,len(u_ele)):                                          # loop that goes over every different elements
                e=label==u_ele[i]                                                 # seperation of the elements
                x_new=x[e]                                                        # select the x values of each element indiviually
                y_new=y[e]
                z_new=z[e]
                c_new=c[e]
                self.ax.scatter3D(x_new,y_new,z_new,c=c_new,label=u_ele[i],s=1)  # scatter plot each element seperatly and assign label
                x_N=len(x) 
                self.ax.set_title('preview (number of atoms={:.0f},     1/{:.0f} of all atoms)'.format(x_N,N))                                                   # assigne x_N value to show the number of plotted atoms in title
            self.canvas.draw_idle()
            apt.set_axes_equal(self.ax)                                            # function for setting the axes equal for 3d plot
            self.ax.legend(loc='center left', bbox_to_anchor=(1.07, 0.5))#, fontsize=7)                                                        # show legends of each element   
            self.ax.set_xlabel('X-axis',fontweight='bold')                     # label x axis
            self.ax.set_ylabel('Y-axis',fontweight='bold')                # label y axis
            self.ax.set_zlabel('Z-axis',fontweight='bold')                     # label z axis
            self.label_succ=Label(text='Sucessfully loaded files!')         # create label for rrng
            self.a_succ=self.canvas_all.create_window(200,260,window=self.label_succ) #placing label on canvas
            self.succ=1
    
    def clear(self):                                                           # function for clearing canvas
        self.fig.clear()                                                       # clearing canvas
        self.canvas.draw_idle()                                                # drawing 
        self.canvas_all.delete('message')                                      # deleting message about concentration data
        
    def plot_tip(self):                                                        # function about plotting tip
        if self.plot_exist==1:                                                 # if plot already exists get the parameters of old plot
            azim=self.ax.azim                                                  # get azimutal angle of old plot
            elev=self.ax.elev
            xlim=self.ax.get_xlim()                                           # get x,y and z limit of old plot
            ylim=self.ax.get_ylim()
            zlim=self.ax.get_zlim()
        if self.i_plot.get()==0:self.invert=0                                 # if inversion checkbox is unmarked set value=0
        else: self.invert=1                                                   # if inversion checkbox is marked set value=1
            
        color_zyl=str(self.cb_color.get())                                    # get color of cylinder from color entry box
        color_cp=str(self.cb_color_cp.get())
        self.canvas_all.delete('message')                                      # clearing message about concentration data
        Z= self.cb_atom.get()                                                  # getting variable from combobox of atom selection
        M = self.slider_res.get()                                              # getting variable from combobox of resolution selection
        SS=self.slider_size.get()                                              # getting variable from combobox of atom size selection
        S=float(SS)/10                                                         # transforming entry into float and deviding by 10 so values 0.1 to 1 are also included
        image_size=self.slider_image_size.get()
        image_size=float(image_size)**2
        u_ele=self.ele
        self.fig.clear()                                                   # if plot single was selected clear all previous plots
        self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[image_size,1],wspace=0.1,hspace=0.1,height_ratios=[image_size,1]) #adjust the size of the figure compared to (111)
        self.ax=self.fig.add_subplot(self.spec[0],projection='3d')         # make new figure with size depending on the image size slider
        if self.plot_exist==1:                                             # if plot already exist set old parameters of plot as new ones
            self.ax.view_init(elev=elev,azim=azim)                         # set azimuthal angle and elevation of old plot as new one
            self.ax.set_xlim3d(xlim[0],xlim[1])                            # set x,y and z limit of old plot as new one
            self.ax.set_ylim3d(ylim[0],ylim[1])
            if self.i_plot.get()==0 and self.invert==0:self.ax.set_zlim3d(zlim[0],zlim[1])  # depending on if the plot should be inverted or not
            if self.i_plot.get()==1 and self.invert==1:self.ax.set_zlim3d(zlim[0],zlim[1])  # reset the z limit of new plot to center it again
        self.plot_exist=1                                                      # set plot exist as one in order to mark that plot exists
        n=len(np.array(self.tip['x']))                                         # get the number of atoms
        N=int(50000/(10**M))                                                   # calc N (the number how many atoms should be plotted)
        if N==0:N=1
        
        x=np.array(self.tip['x'])[::N]                                         # select x cooridant (only take every Nth atom to plot later)
        y=np.array(self.tip['y'])[::N]                                         # select y cooridant from tip data
        z=np.array(self.tip['z'])[::N]                                         # select z cooridant from tip data
        c=np.array(self.tip['colour'])[::N]                                    # select the colour column in the tip data
        label=np.array(self.tip['comp'])[::N]                                  # select the composition column in the tip data

        max_x=max(x)
        min_x=min(x)
        max_z=max(z)
        min_z=min(z)
        max_y=max(y)
        min_y=min(y)
        for i in range(0,len(u_ele)):                                          # loop that goes over every different elements
            e=label==u_ele[i]                                                 # seperation of the elements
            x_new=x[e]                                                        # select the x values of each element indiviually
            y_new=y[e]
            z_new=z[e]
            c_new=c[e]     
            if self.i_plot.get()==1: z_new=-z_new                                          # if inverse tip is selected change the sign of the z coordinates
            if Z=='all':                                                      # if all was selected plot all of them together
                X=np.array([x,y,z]).T 
                c_new[c_new=='#FFFFFF']='#000000'
                self.ax.scatter(x_new,y_new,z_new,c=c_new,label=u_ele[i],s=S,zorder = 1,depthshade = False,picker=True)  # scatter plot each element seperatly and assign label
                x_N=len(x) 
                self.ax.set_title('number of atoms={:.0f},     1/{:.0f} of all atoms'.format(x_N,N))                                                   # assigne x_N value to show the number of plotted atoms in title
            elif u_ele[i]==Z:                                                 # if a certain element was selected in combobox for element selection
                X=np.array([x_new,y_new,z_new]).T                             # putting all 
                c_new[c_new=='#FFFFFF']='#000000'
                self.ax.scatter(X[:,0],X[:,1],X[:,2],c=c_new,label=u_ele[i],s=S,zorder = 1, depthshade = False,picker=True)  #scatter plot each element seperatly and assign label         
                x_N=len(x_new)                                                 # assigne x_N only to show the number of plotted atoms of selected element 
                self.ax.set_title('number of atoms={:.0f},     1/{:.0f} of all atoms of this type'.format(x_N,N))                                                   
        apt.set_axes_equal(self.ax)                                            # function for setting the axes equal for 3d plot
        self.ax.legend(loc='center left', bbox_to_anchor=(1.07, 0.5))#, fontsize=7)                                                       # show legends of each element
        if self.var_axis.get()==0:                                             # if show axis was selected
            self.ax.set_xlabel('X-axis',fontweight='bold')                     # label x axis
            self.ax.set_ylabel('Y-axis',fontweight='bold')                     # label y axis
            self.ax.set_zlabel('Z-axis',fontweight='bold')                     # label z axis
        elif self.var_axis.get()==1:                                           # if show axis was not selected
            self.ax.set_axis_off()                                             # dont show axis
        
        if self.var_cyl.get()==1:                                              # if show clyinder was selected
            x_pos=self.cly_x.get()                                            # get x value from x entry box
            if x_pos=='': x_pos=0
            else:x_pos = float(x_pos)
            y_pos =self.cly_y.get()                                           # get y value from y entry box
            if y_pos=='': y_pos=0
            else:y_pos = float(y_pos)
            z_pos = self.cly_z.get()                                          # get z value from z entry box
            if z_pos=='': z_pos=0
            else:z_pos = float(z_pos) 
            if self.i_plot.get()==1:      z_pos = -z_pos                      # if inverse tip is selected inverse the z coordinates of the cylinder              
            height = float(self.height.get())                                 # get height value from height entry box
            alpha = float(self.alpha.get())                                   # get tilt along y value from alpha entry box
            beta = float(self.beta.get())                                     # get tilt along x value from beta entry box
            r = float(self.radius.get())                                      # get radius value from radius entry box 
            theta=np.linspace(0,2*np.pi,201)                                  # create linspace variable for plotting the circle
            alpha=math.radians(alpha)                                         # transform alpha from deg in rad
            beta=math.radians(beta)                                           # transfrom beta from deg in rad
            Xc,Yc,Zc=data_for_cylinder_along_z(0,0,0,r,height)  # calculating cylinder
            Z=np.array([Xc,Yc,Zc]).T
            matrix_l=np.array([(np.cos(beta),                   np.sin(alpha)*np.sin(beta),        np.cos(alpha)*np.sin(beta)),
                               (0,                              np.cos(alpha),                     -np.sin(alpha)),
                               (-np.sin(beta),                  np.sin(alpha)*np.cos(beta),        np.cos(alpha)*np.cos(beta))])
            pro=[]
            for j in range(0,len(Z)):
                 for i in range (0,len(Z[j])):
                         pro.append(matrix_l.dot(Z[i][j]))  
            pro=np.array(pro)                
            Z_new=pro.reshape(50,50,3).T
            self.ax.plot_surface(Z_new[0]+x_pos,Z_new[1]+y_pos,Z_new[2]+z_pos,alpha=0.4,color=color_zyl) # plotting cylinder
            apt.set_axes_equal(self.ax)
            
        if self.var_points.get()==1:
            XX=[]
            YY=[]
            ZZ=[]
            points=[]
            points=self.text_box.get(1.0,END)
            points=points.split('\n')
            points=[x for x in points if x!='']
            for i in range(0,len(points)):
                points_inner=points[i].split(',')
                xyz2=[]
                for j in points_inner:
                    m=j.split('=')
                    xyz2.append(m[1])
                x2=float(xyz2[0])
                y2=float(xyz2[1])
                z2=float(xyz2[2])
                XX.append(x2)
                YY.append(y2)
                ZZ.append(z2)
                if self.i_plot.get()==1:z2=-z2                                          # if inverse tip is selected
                self.ax.scatter3D(x2,y2,z2,marker='+',color=color_cp,s=S*100)
            if len(XX)>=3 and self.var_plane.get()==1:
                center_x=sum(XX)/len(XX)
                center_y=sum(YY)/len(YY)
                center_z=sum(ZZ)/len(ZZ)
                self.cly_x=StringVar(value=center_x)
                self.cly_y=StringVar(value=center_y)
                self.cly_z=StringVar(value=center_z)
                tmp_C=[]
                for i in range(len(XX)):
                    tmp_C.append([XX[i],YY[i],ZZ[i]])
                C=np.array(tmp_C).T
                svd=np.linalg.svd(C - np.mean(C, axis=1, keepdims=True))
                left=svd[0]
                fit=left[:,-1]
                #print('solution: %f x +%f y +%f z = 0' %(fit[0],fit[1],fit[2]))
                normal=np.array([fit[0],fit[1],fit[2]])   
                r=5
                self.ax.plot([center_x,center_x+r*normal[0]],[center_y,center_y+r*normal[1]],[center_z,center_z+r*normal[2]],color='r',linewidth=5)
                xlim=self.ax.get_xlim()
                ylim=self.ax.get_ylim()
                X,Y=np.meshgrid(np.arange(xlim[0],xlim[1]),np.arange(ylim[0],ylim[1]))
                Z=np.zeros(X.shape)
                for r in range(X.shape[0]):
                    for c in range(X.shape[1]):
                        #Z[r,c]=fit[0]*X[r,c]+fit[1]*Y[r,c]+fit[2]
                        Z[r,c]=-(fit[0]*X[r,c]+fit[1]*Y[r,c])/fit[2]
                #self.ax.plot([cpx,fit[0]],[cpy,fit[1]],[cpz,-1])
                self.ax.plot_wireframe(X+center_x,Y+center_y,Z+center_z,color='k')
        self.canvas.draw_idle()  
  
        def distance(point, event):
#         Return distance between mouse position and given data point
#         Args:point (np.array): np.array of shape (3,), with x,y,z in data coords event (MouseEvent): mouse event (which contains mouse position in .x and .xdata)
#         Returns:distance (np.float64): distance (in screen coords) between mouse pos and data point          
             assert point.shape == (3,), "distance: point.shape is wrong: %s, must be (3,)" % point.shape  
             x2, y2, _ = proj3d.proj_transform(point[0], point[1], point[2], self.ax.get_proj())   # Project 3d data space to 2d data space
             x3, y3 = self.ax.transData.transform((x2, y2))                                        # Convert 2d data space to 2d screen space
             return np.sqrt ((x3 - event.x)**2 + (y3 - event.y)**2)                                # calculate distance
    
        def onclick(event):
            distances = [distance (X[i,0:3], event) for i in range(X.shape[0])]                                              # use function distance to evaluate the closests index to mouse click
            index=np.argmin(distances)                                                                                       # select the closest index
            self.ax.scatter(X[index, 0], X[index, 1], X[index, 2],marker='+',color=color_cp,s=S*100)                         # plot controlpoint at closest index point
            text='x='+str(np.round(X[index, 0],4))+', y='+str(np.round(X[index, 1],4))+', z='+str(np.round(X[index, 2],4))   # write x,y,z coordinates into text file
            if self.i_plot.get()==1:text='x='+str(np.round(X[index, 0],4))+', y='+str(np.round(X[index, 1],4))+', z='+str(-np.round(X[index, 2],4))
            self.text_box.insert(END,text)                                                                                   # insert text file in textbox
            self.text_box.insert(END,'\n')                                                                                   # make sure next control point is next row
            self.canvas.draw_idle()
                
        if self.control.get()==1: 
            if self.diss==1:self.fig.canvas.mpl_disconnect(self.cid)
            self.cid=self.fig.canvas.mpl_connect('button_press_event',onclick)
            self.diss=1
        elif self.control.get()==0 and self.diss==1: 
            self.fig.canvas.mpl_disconnect(self.cid)
            self.diss=0
          
    def calc_con(self):                                                        # function for calculating the concentration
        if self.diss2==1:self.fig.canvas.mpl_disconnect(self.cid) 
        self.plot_exist=0
        self.print_con=[]
        self.print_con2=[]
        self.canvas_all.delete('message') 
        start_end=self.cb_bulk.get()
        atom= self.cb_atom.get()
        u_ele_real=self.ele
        image_size=self.slider_image_size.get()
        image_size=float(image_size)/2
        self.fig.clear()
        self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[image_size,1],wspace=0.5,hspace=0.5,height_ratios=[image_size,1]) #adjust the size of the figure compared to (111)
        self.ax1=self.fig.add_subplot(self.spec[0])
        self.ax1.set_xlabel('z position of cylinder')
        self.ax1.set_ylabel('concentration [%]')
        x_real=np.array(self.tip['x'])
        y_real=np.array(self.tip['y'])
        z_real=np.array(self.tip['z'])
        if self.i_plot.get()==1:                                     # atoms z coordinatines inversion
            z_real = -z_real
        c_real=np.array(self.tip['colour'])
        label_real=np.array(self.tip['comp'])
        inter=float(self.slider_inter.get()/20)
        height = float(self.height.get())
        alpha = float(self.alpha.get())
        beta = float(self.beta.get())
        r = float(self.radius.get())
        x_pos = float(self.cly_x.get())
        y_pos = float(self.cly_y.get())
        z_pos = float(self.cly_z.get())
        if self.i_plot.get()==1:                                     # cylinder z coordinatines inversion
            z_pos = -z_pos
        z_start=-height/2
        z_end=height/2
        alpha=math.radians(-alpha)
        beta=math.radians(-beta)
        ###################   tilt and move zylinder ######################################
        pro=[]
        Z=np.array([x_real-x_pos,y_real-y_pos,z_real-z_pos]).T
        M_ROT1=[(1,      0,                  0     ),            # rotational matrix along the x-direction
                (0,  np.cos(alpha),  -np.sin(alpha)),
                (0,  np.sin(alpha),    np.cos(alpha))]

        M_ROT2=[(np.cos(beta),      0,       np.sin(beta)),     # rotational matrix along the y-direction
                (0,                  1,            0      ),
                (-np.sin(beta),      0,       np.cos(beta))]
        matrix_l=np.array(M_ROT1).dot(M_ROT2)
        for j in range(0,len(Z)):
                    pro.append(matrix_l.dot(Z[j])) 
        Z_new=np.array(pro)
        x_real=Z_new[:,0]
        y_real=Z_new[:,1]
        z_real=Z_new[:,2]
        ######################################################################### 
        xy=((x_real)**2+(y_real)**2)**(1/2)                                    # cirlce constraint for atoms       
        circle=xy<=r                                                           # only select atoms inside of circle
        message='concentration: \n'                                            # initialize the message displayed on the right side of the plot
        for m in range(0,len(u_ele_real)):                                     # loop over each individual elements
            plot_z=[]                                                          # x component of concentration plots 
            Num=[]                                                             # Number of atoms inside the circle and intervals (for concentration calculation)
            Num_all=[]                                                         # total number of atoms (of each element) to calculate the % contribution of each
            e=label_real[circle]==u_ele_real[m]                                # seperation of the elements
            z_circle=z_real[circle]                                            # only selec the atoms with circle constraint
            z_p=z_circle[e]                                                    # only select the atoms wth circle constraint of specific element
            c_circle=c_real[circle]
            c_p=c_circle[e]
            for j in range (0,int((z_end-z_start)/inter)+1):                     # loop of intervals
                init=z_p>=z_start+j*inter                                      # starting condition of interval for each element
                init_all=z_circle>=z_start+j*inter                             # starting condition of interval for all elements together
                z_all2=z_circle[init_all]                                      # selecting atoms which fulfill starting constraint for all elements
                z_p2=z_p[init]                                                 # selecting atoms which fulfill the starting constraint for each individual element
                init2=z_p2<=z_start+(j*inter+inter)                            # ending constraint of interval
                init2_all=z_all2<=z_start+(j*inter+inter)                      # ending constraint of interval for all elements
                z_all3=z_all2[init2_all]                                       # selecting atoms which also fulfill ending constraint for all elements
                z_p3=z_p2[init2]                                               # selecting atom which also fulfill ending constraint for each individual elements
                Num_all.append(len(z_all3))                                    # append the number of atoms which fulfill both contraints
                Num.append(len(z_p3))                                          # append the number of atoms which fulfill both constraints for each individual element
                plot_z.append(z_start+j*inter)#+inter/2)                       # append the x-coordinate for the concentration plot
            a=np.array(Num,dtype=float)                                        # transform total number of atoms for each individual element into array of floats
            b=np.array(Num_all,dtype=float)                                    # transform total number of atoms into array of floats
            if self.var_switch.get()==1: 
                if start_end=='start':b=np.array(Num_all[0])
                last=len(np.array(Num_all))-1    
                if start_end=='end':b=np.array(Num_all[last])
                if b==0:messagebox.showinfo(title='APT_analyzer',message='Reference bulk concentration is 0')
            self.conc=100*np.divide(a,b, out=np.zeros_like(a), where=b!=0)     # calculate concentration, if total amount of atoms == 0, result is 0
            message=message+u_ele_real[m]                                      # append element type into message
            message=message+': maximal value: '                                # add text to message
            message=message+str(round(max(self.conc),2))                            # add maximum of concentration to message
            message=message+' minimal value: '                                 # add text to message
            message=message+str(round(min(self.conc),2))                            # add minimum of concentration to message
            message=message+'\n'                                               # add line split to message  
            if atom=='all':                                                       # seperate if atom select checkbox 'all' is selected
                if c_p[0]=='#FFFFFF':c_p[0]='#000000'                                        # if color of particles is white change it to black so it can be seen with white background
                self.ax1.plot(plot_z,self.conc,c_p[0],label=u_ele_real[m])                 # plot the concentration of each element individually
                self.print_con2.append(u_ele_real[m])
                self.print_con.append(self.conc)
            elif u_ele_real[m]==atom:                                             # if certain element is selected in checkbox
                if c_p[0]=='#FFFFFF':c_p[0]='#000000'
                self.ax1.plot(plot_z,self.conc,c_p[0],label=u_ele_real[m])                 # only plot selected element from the select atom checkbox
                self.print_con2.append(u_ele_real[m])
                self.print_con.append(self.conc)
        self.print_con2.append('z_cyl')
        plot_z2=plot_z.copy()
        plot_z2=np.array(plot_z2).reshape(np.shape(np.array(self.conc)))
        self.print_con.append(plot_z2)
        self.df_con=pd.DataFrame(np.array(self.print_con).T,columns=self.print_con2)
        self.ax1.legend(loc='center left', bbox_to_anchor=(1.07, 0.5))
        self.ax1.set_title('concentration of elements along cylinder / intervalsize=%1.2f' %inter)
        self.canvas.draw_idle()
        self.message=Text(height = 20,width = 50,relief=FLAT)
        self.message.insert(END, message)
        self.message.configure(state='disabled')
        image_ratio=image_size/2.5
        place_factor=0.2
        place_value=1450-place_factor*(1450-image_ratio*1450)
        self.canvas_all.create_window(place_value,300,window=self.message,tags='message')
        self.conc_check=1
        
        
        
    def calc_excess(self):
        self.plot_exist=0
        self.print_ex=[]
        self.print_ex2=[]
        self.canvas_all.delete('message')  
        atom= self.cb_atom.get()      
        color_zyl=str(self.cb_color.get())
        u_ele_real=self.ele        
        image_size=self.slider_image_size.get()
        image_size=float(image_size)/2
        self.fig.clear()
        self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[image_size,1],wspace=0.5,hspace=0.5,height_ratios=[image_size,1]) #adjust the size of the figure compared to (111)
        self.ax1=self.fig.add_subplot(self.spec[0])
        self.ax1.set_xlabel('z position of cylinder')
        self.ax1.set_ylabel('Total amount of atoms')
        x_real=np.array(self.tip['x'])
        y_real=np.array(self.tip['y'])
        z_real=np.array(self.tip['z'])                             # atoms z coordinatines inversion
        if self.i_plot.get()==1:
            z_real=-z_real
        c_real=np.array(self.tip['colour'])
        label_real=np.array(self.tip['comp'])
        inter=float(self.slider_inter.get()/20)
        height = float(self.height.get())
        alpha = float(self.alpha.get())
        beta = float(self.beta.get())
        r = float(self.radius.get())
        x_pos = float(self.cly_x.get())
        y_pos = float(self.cly_y.get())
        z_pos = float(self.cly_z.get())
        if self.i_plot.get()==1:                                   # cylinder z coordinatines inversion
            z_pos=-z_pos
        z_start=-height/2
        z_end=height/2
        alpha=math.radians(-alpha)                               # transfroms radius into radians
        beta=math.radians(-beta)                                 # - because otherwise it tilts the cylinder in the wrong direction (dont know why)
        ###################   tilt and move zylinder ######################################
        pro=[]
        Z=np.array([x_real-x_pos,y_real-y_pos,z_real-z_pos]).T
        M_ROT1=[(1,      0,                  0     ),            # rotational matrix along the x-direction
                (0,  np.cos(alpha),  -np.sin(alpha)),
                (0,  np.sin(alpha),    np.cos(alpha))]

        M_ROT2=[(np.cos(beta),      0,       np.sin(beta)),     # rotational matrix along the y-direction
                (0,                  1,            0      ),
                (-np.sin(beta),      0,       np.cos(beta))]
        matrix_l=np.array(M_ROT1).dot(M_ROT2)
        for j in range(0,len(Z)):
                    pro.append(matrix_l.dot(Z[j])) 
        Z_new=np.array(pro)
        x_real=Z_new[:,0]
        y_real=Z_new[:,1]
        z_real=Z_new[:,2]
        ######################################################################### 
        xy=((x_real)**2+(y_real)**2)**(1/2)
        circle=xy<=r
        for m in range(0,len(u_ele_real)):
            plot_z=[]
            excess=[]
            ex=0
            e=label_real[circle]==u_ele_real[m]               #seperation of the atoms 
            z_circle=z_real[circle]
            z_p=z_circle[e]
            c_circle=c_real[circle]
            c_p=c_circle[e]
            for j in range (0,int((z_end-z_start)/inter)+1):
                init=z_p>=z_start+j*inter   
                z_p2=z_p[init]
                init2=z_p2<=z_start+(j*inter+inter)
                z_p3=z_p2[init2]
                ex+=len(z_p3)
                excess.append(ex)
                plot_z.append(z_start+j*inter)#+inter/2)   
            if self.var_norm.get()==1:excess=np.array(excess)/(max(excess))
            if atom=='all':
                if c_p[0]=='#FFFFFF':c_p[0]='#000000'
                self.ax1.plot(plot_z,excess,c_p[0],label=u_ele_real[m])
                excess_save=excess.copy()
                self.print_ex2.append(u_ele_real[m])
                self.print_ex.append(np.array(excess))
            elif u_ele_real[m]==atom:
                if c_p[0]=='#FFFFFF':c_p[0]='#000000'
                self.ax1.plot(plot_z,excess,c_p[0],label=u_ele_real[m]) 
                excess_save=excess.copy()
                self.print_ex2.append(u_ele_real[m])
                self.print_ex.append(np.array(excess)) 
        self.print_ex2.append('z_cyl')
        plot_z2=plot_z.copy()
        plot_z2=np.array(plot_z2).reshape(np.shape(np.array(excess)))
        self.print_ex.append(plot_z2)
        self.df_ex=pd.DataFrame(np.array(self.print_ex).T,columns=self.print_ex2)
        self.ax1.set_title('Excess (total atom number of elements along cylinder) / intervalsize=%1.2f ' %inter)
        self.ax1.legend(loc='center left', bbox_to_anchor=(1.07, 0.5))
        
        points_ex=self.text_box_ex.get(1.0,END)   # get text from text box
        if points_ex!='\n':
            points_ex=points_ex.split('\n')           # split the text into each row
            points_ex=[x for x in points_ex if x!=''] # remove any rows that dont contain information
            XX=[]
            YY=[]
            for i in range(0,len(points_ex)):      # loop over all rows
                    points_inner=points_ex[i].split(',')   # split each row into parts of each coordinates
                    xy2=[]
                    for j in points_inner:               # loop over all values inside row
                        m=j.split('=')                   # isolate actual number
                        xy2.append(m[1])                # save number
                    XX.append(float(xy2[0]))            # save x,y and z coordinate
                    YY.append(float(xy2[1]))
            if len(XX)==4:
                k1=(YY[0]-YY[1])/(XX[0]-XX[1])
                k3=(YY[2]-YY[3])/(XX[2]-XX[3])
                d1=YY[1]-k1*XX[1]
                d3=YY[3]-k3*XX[3]
                x=np.array([-height/2,height/2])
                y1=gerade(np.array(k1),x,np.array(d1))
                y3=gerade(np.array(k3),x,np.array(d3))
                diff=[]
                for i in range(0,len(excess_save)):
                    upper_line=gerade(np.array(k3),plot_z[i],np.array(d3))
                    lower_line=gerade(np.array(k1),plot_z[i],np.array(d1))  
                    diff.append(np.abs((upper_line-excess_save[i])-(excess_save[i]-lower_line)))
                plot_z=np.array(plot_z)
                x_found=plot_z[diff==min(diff)]
                y_excess_u=gerade(np.array(k3),x_found,np.array(d3))
                y_excess_l=gerade(np.array(k1),x_found,np.array(d1))
                excess_atom_value=y_excess_u[0]-y_excess_l[0]
                excess_value=excess_atom_value/(np.pi*r**2)
                self.ax1.plot([x_found,x_found],[y_excess_l,y_excess_u])
                self.ax1.plot(x,y1,color=color_zyl)
                self.ax1.plot(x,y3,color=color_zyl)
                self.ax1.scatter(XX,YY,color='black',marker='x')
                self.ax1.annotate(" 1", (XX[0], YY[0]))
                self.ax1.annotate(" 2", (XX[1], YY[1]))
                self.ax1.annotate(" 3", (XX[2], YY[2]))
                self.ax1.annotate(" 4", (XX[3], YY[3]))
                self.output = Text(height = 10,width = 50,relief=FLAT)
                image_ratio=image_size/2.5
                place_factor=0.25
                place_value=650-place_factor*(650-image_ratio*650)
                self.canvas_all.create_window(650,place_value,window=self.output,tags='message')
                message2='Calculated with the following 4 points: \n'
                message2=message2+'point 1: x = '+str(XX[0])+'   y = '+str(YY[0])+'\n'
                message2=message2+'point 2: x = '+str(XX[1])+'   y = '+str(YY[1])+'\n'
                message2=message2+'point 3: x = '+str(XX[2])+'   y = '+str(YY[2])+'\n'
                message2=message2+'point 4: x = '+str(XX[3])+'   y = '+str(YY[3])+'\n'+'\n'
                message2=message2+'the interfacial excess atoms = '+str(np.round(excess_atom_value,3))+' atoms'+'\n'
                message2=message2+'the interfacial excess = '+str(np.round(excess_value,3))+' atoms/nm²'
                self.output.insert(END, message2)
                self.output.configure(state='disabled')
                self.canvas.draw_idle()    
            elif len(XX)>=4:messagebox.showinfo(title='APT_analyzer',message='Too many fit points selected. Reduce to 4 points to fit excess.')
        def onclick(event):
            x=np.round(event.xdata,2)
            y=np.round(event.ydata,2)
            self.ax1.scatter(x,y,color='black',marker='x')
            text='x='+str(x)+', y='+str(y)
            self.text_box_ex.insert(END,text)
            self.text_box_ex.insert(END,'\n')
            self.canvas.draw_idle()   
        if self.diss2==1:self.fig.canvas.mpl_disconnect(self.cid)    
        self.cid=self.fig.canvas.mpl_connect('button_press_event',onclick)
        self.diss2=1
        self.canvas.draw_idle()  
        self.excess_check=1
                    
    def calc_zoom(self):     
        self.plot_exist=0
        self.canvas_all.delete('message')   
        image_size=self.slider_image_size.get()
        image_size=float(image_size)**2
        self.fig.clear()
        self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[image_size,1],wspace=0.5,hspace=0.5,height_ratios=[image_size,1]) #adjust the size of the figure compared to (111)
        self.ax2=self.fig.add_subplot(self.spec[0],projection='3d')
        atom= self.cb_atom.get()
        M = self.slider_res.get()
        SS=self.slider_size.get()
        S=float(SS)/10
        u_ele_real=self.ele
        n=len(np.array(self.tip['x']))
        N_plot=int(50000/(10**M))
        if N_plot==0:N_plot=1
        x_real=np.array(self.tip['x'])
        y_real=np.array(self.tip['y'])    
        z_real=np.array(self.tip['z'])
        if self.i_plot.get()==1:                                # atoms z coordinatines inversion
            z_real = -z_real
        c_real=np.array(self.tip['colour'])
        label_real=np.array(self.tip['comp'])
        height = float(self.height.get())
        alpha = -float(self.alpha.get())
        beta = -float(self.beta.get())
        r = float(self.radius.get())
        x_pos = float(self.cly_x.get())
        y_pos = float(self.cly_y.get())
        z_pos = float(self.cly_z.get())
        if self.i_plot.get()==1:                                # cylinder z coordinatines inversion
            z_pos = -z_pos
        color_zyl=str(self.cb_color.get())
        z_start=-height/2
        z_end=height/2
        alpha=math.radians(alpha)
        beta=math.radians(beta)
        ###################   tilt and move zylinder ######################################
        pro=[]
        Z=np.array([x_real-x_pos,y_real-y_pos,z_real-z_pos]).T
        M_ROT1=[(1,      0,                  0     ),            # rotational matrix along the x-direction
                (0,  np.cos(alpha),  -np.sin(alpha)),
                (0,  np.sin(alpha),    np.cos(alpha))]

        M_ROT2=[(np.cos(beta),      0,       np.sin(beta)),     # rotational matrix along the y-direction
                (0,                  1,            0      ),
                (-np.sin(beta),      0,       np.cos(beta))]
        matrix_l=np.array(M_ROT1).dot(M_ROT2)
        for j in range(0,len(Z)):
                    pro.append(matrix_l.dot(Z[j])) 
        Z_new=np.array(pro)
        x_real=Z_new[:,0]
        y_real=Z_new[:,1]
        z_real=Z_new[:,2]
        #########################################################################   
        xy=((x_real)**2+(y_real)**2)**(1/2)
        circle=xy<=r
        for m in range(0,len(u_ele_real)):
            e=label_real[circle]==u_ele_real[m]               #seperation of the atoms 
            z_circle=z_real[circle]
            x_circle=x_real[circle]
            y_circle=y_real[circle]
            c_circle=c_real[circle]
            z_p=z_circle[e]  
            x_p=x_circle[e] 
            y_p=y_circle[e]
            c_p=c_circle[e]
            init_plot=z_p>=z_start
            x_pl2=x_p[init_plot]
            y_pl2=y_p[init_plot]
            z_pl2=z_p[init_plot]
            c_pl2=c_p[init_plot]
            init_plot2=z_pl2<=z_end    
            x_pl3=x_pl2[init_plot2]
            y_pl3=y_pl2[init_plot2]
            z_pl3=z_pl2[init_plot2]
            c_pl3=c_pl2[init_plot2]
            if atom=='all':
                c_pl3[c_pl3=='#FFFFFF']='#000000'
                x_N=len(x_real[::N_plot])
                self.ax2.scatter3D(x_pl3[::N_plot],y_pl3[::N_plot],z_pl3[::N_plot],c=c_pl3[::N_plot],label=u_ele_real[m],s=S)
                self.ax2.set_title('number of atoms={:.0f},     1/{:.0f} of all atoms'.format(x_N,N_plot)) 
            elif u_ele_real[m]==atom:
                c_pl3[c_pl3=='#FFFFFF']='#000000'
                x_N=len(x_pl3[::N_plot])
                self.ax2.scatter3D(x_pl3[::N_plot],y_pl3[::N_plot],z_pl3[::N_plot],c=c_pl3[::N_plot],label=u_ele_real[m],s=S)
                self.ax2.set_title('number of atoms={:.0f},     1/{:.0f} of all atoms of this type'.format(x_N,N_plot))                  
        if self.var_axis.get()==0: 
            #self.ax2.set_title('number of atoms=%i' %x_N)
            self.ax2.set_xlabel('X-axis',fontweight='bold')
            self.ax2.set_ylabel('Y-axis',fontweight='bold')
            self.ax2.set_zlabel('Z-axis',fontweight='bold')
        elif self.var_axis.get()==1:
            self.ax2.set_axis_off()         
        apt.set_axes_equal(self.ax2)   
        self.ax2.legend() 
        if self.var_cyl.get()==1:
            x_pos = float(self.cly_x.get())
            y_pos = float(self.cly_y.get())
            z_pos = float(self.cly_z.get())
            height = float(self.height.get())
            alpha = float(self.alpha.get())
            beta = float(self.beta.get())
            r = float(self.radius.get())
            color_zyl=str(self.cb_color.get())
            theta=np.linspace(0,2*np.pi,201)
            z_start=-height/2
            z_end=height/2
            alpha=math.radians(alpha)
            beta=math.radians(beta)
        
            y_circle=r*np.cos(theta)
            x_circle=r*np.sin(theta)
            z_circle_s=np.ones(201)*z_start
            z_circle_e=np.ones(201)*z_end

            x_line1=np.ones(201)*r
            x_line2=np.ones(201)*-r
            y_line=np.ones(201)*0
            z_line=np.linspace(z_start,z_end,201)
            lw=5
            self.ax2.plot(x_line1,y_line,z_line,color_zyl,linewidth=lw)
            self.ax2.plot(x_line2,y_line,z_line,color_zyl,linewidth=lw)
            self.ax2.plot(x_circle,y_circle,z_circle_s,color_zyl,linewidth=lw)  
            self.ax2.plot(x_circle,y_circle,z_circle_e,color_zyl,linewidth=lw) 
        self.canvas.draw_idle() 

tk = Tk()                                                                      # tkinter
tk.title("APT analyzer")                                                       # select title of tool
tk.geometry('1100x700')                                                        # select size of tool
tt=APT_analyzer(tk)                                                            # select tool              
tk.mainloop()                                                                  # start tool